In [70]:
import requests
import pandas as pd
import time

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:28.0) Gecko/20100101 Firefox/28.0',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}

In [8]:
resp = requests.get('https://dev-mobileapp.bigc.vn/api/store/get-list-store?apiclientid=3693146&token=a09fcbfe4ec1edb8f3b2214076a3093d&sign=1f664a32aa64aa3f66fa44da534ef139',headers=headers)

In [15]:
store_data = pd.DataFrame(resp.json()['message'])

In [16]:
store_data.to_csv('bigc_store_data.csv')

In [36]:
resp = requests.get('https://dev-mobileapp.bigc.vn/api/order2/ListCategory?apiclientid=3693146&token=a09fcbfe4ec1edb8f3b2214076a3093d&sign=1f664a32aa64aa3f66fa44da534ef139&page=2&store=108',headers=headers)

In [37]:
category_info = pd.DataFrame(resp.json()['products'])

In [38]:
category_info.to_csv('bigc_category_data.csv',mode='a',header=None,encoding='utf-8-sig')

In [54]:
pay_load = {"category":7,"hourDelivery":0,"page":9,"store":108}
resp = requests.post('https://dev-mobileapp.bigc.vn/api/order2/listProduct?apiclientid=3693146&token=a09fcbfe4ec1edb8f3b2214076a3093d&sign=1f664a32aa64aa3f66fa44da534ef139',headers=headers,data = pay_load)

In [62]:
resp.json()['pagination']

{'total_pages': 9, 'page_size': 15, 'current_page': 9}

In [72]:
store_id = 108
cate_list = category_info['id']
product = pd.DataFrame()
for cate in cate_list:
    start_page = 1
    pay_load = {"category":int(cate),"hourDelivery":0,"page":start_page,"store":int(store_id)}
    resp = requests.post('https://dev-mobileapp.bigc.vn/api/order2/listProduct?apiclientid=3693146&token=a09fcbfe4ec1edb8f3b2214076a3093d&sign=1f664a32aa64aa3f66fa44da534ef139',headers=headers,data = pay_load)
    df_tmp = pd.DataFrame(resp.json()['products'])
    last_page = resp.json()['pagination']['total_pages']
    product = pd.concat([product,df_tmp])
    if start_page < last_page:
        start_page = start_page+1
    try:
        for next_page in range(start_page,last_page+1,1):
            pay_load = {"category":int(cate),"hourDelivery":0,"page":start_page,"store":int(store_id)}
            resp = requests.post('https://dev-mobileapp.bigc.vn/api/order2/listProduct?apiclientid=3693146&token=a09fcbfe4ec1edb8f3b2214076a3093d&sign=1f664a32aa64aa3f66fa44da534ef139',headers=headers,data = pay_load)
            df_tmp = pd.DataFrame(resp.json()['products'])
            product = pd.concat([product,df_tmp])
    except:
        print(f'error on {cate}, payload: {pay_load}')
        time.sleep(5)
        continue
product.to_csv('big_product.csv',encoding='utf-8-sig')

In [69]:
pay_load

{'category': 21, 'hourDelivery': 0, 'page': 2, 'store': 108}